<a href="https://colab.research.google.com/github/pedrootrevisan/SensoriamentoRemoto/blob/main/NBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import ee
import geemap
import pandas

In [5]:
ee.Authenticate()
ee.Initialize(project = 'ee-pedrotrevisanoliveira')


In [6]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [13]:
#area de interesse
roi = ee.Geometry.Point([-49.66, -18.94]);

In [14]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                                  .filterBounds(roi)\
                                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1))\
                                  .filterDate('2020-01-01','2021-01-01')
print('qtd informações',collection.size().getInfo())

qtd informações 21


In [15]:
melhor_imagem = collection.sort('CLOUDY_PIXEL_PERCENTAGE',True).first()
print('ID', melhor_imagem.get('PRODUCT_ID').getInfo())
print('% nuvens',melhor_imagem.get('CLOUDY_PIXEL_PERCENTAGE').getInfo())

image_scale = melhor_imagem.select('B.*').multiply(0.0001)


ID S2B_MSIL2A_20200904T133229_N0214_R081_T22KFE_20200908T100034
% nuvens 0.006269


In [16]:
map.addLayer(image_scale,{'bands':['B4','B3','B2'],'min':0.019, 'max':0.19},'RGB Sentinel')
map.addLayer(image_scale,{'bands':['B8','B4','B3'],'min':0.019, 'max':0.19},'False Color')
map

Map(bottom=1161440.0, center=[-18.96668845695318, -49.64960998032163], controls=(WidgetControl(options=['posit…

In [17]:
nbr = image_scale.normalizedDifference(['B8','B12']).rename('nbr')
map.addLayer(nbr,{'palette': ['red','orange','yellow','green','darkgreen'],'min':0,'max':0.8},'NBR')
map.addLayer(roi.buffer(5000),{},'ROI')
map

Map(bottom=1161440.0, center=[-18.96668845695318, -49.64960998032163], controls=(WidgetControl(options=['posit…

In [20]:
#baixar imagem
task = ee.batch.Export.image.toDrive(image=nbr,
                                     region = roi.buffer(1000),
                                     description='NBR',
                                     folder='Imagens',
                                     fileNamePrefix='NBR',
                                     scale=20,
                                     maxPixels=1e13,
                                     crs='EPSG:4326')
task.start()

In [21]:
task.status()

{'state': 'RUNNING',
 'description': 'NBR',
 'priority': 100,
 'creation_timestamp_ms': 1718633173005,
 'update_timestamp_ms': 1718633187038,
 'start_timestamp_ms': 1718633177946,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'O4ZAE6EC6CGKLGH2DZ66P7MF',
 'name': 'projects/ee-pedrotrevisanoliveira/operations/O4ZAE6EC6CGKLGH2DZ66P7MF'}